In [1]:
from datetime import datetime
import pandas as pd

def clean_data(df):
    # Round column 'Tax 5%' (Number of decimals: 2)
    df = df.round({'Tax 5%': 2})
    # Round column 'Total' (Number of decimals: 2)
    df = df.round({'Total': 2})
    # Round column 'cogs' (Number of decimals: 2)
    df = df.round({'cogs': 2})
    # Round columns 'gross margin percentage', 'gross income' (Number of decimals: 2)
    df = df.round({'gross margin percentage': 2, 'gross income': 2})
    # Derive column 'Date_new' from column: 'Date'
    def Date_new(Date):
        """
        Transform based on the following examples:
           Date            Output
        1: "01/05/2019" => "01/05/2019"
        2: "03/08/2019" => "03/08/2019"
        3: "03/03/2019" => "03/03/2019"
        4: "1/27/2019"  => "01/27/2019"
        """
        enum1 = [i for i in range(len(Date)) if Date.startswith("/", i)]
        find1 = Date.find("/")
        index1 = enum1[1] + 2
        index2 = enum1[1] + 3
        return Date[index1:index2] + Date[find1 - 1:find1 + 4] + Date.split("/")[-1]
    df.insert(11, "Date_new", df.apply(lambda row : Date_new(row["Date"]), axis=1))
    # Drop column: 'Date'
    df = df.drop(columns=['Date'])
    # Rename column 'Date_new' to 'Date'
    df = df.rename(columns={'Date_new': 'Date'})
    # Derive column 'Date_new' from column: 'Date'
    def Date_new(Date):
        """
        Transform based on the following examples:
           Date            Output
        1: "01/05/2019" => "2019-01-05"
        """
        date1 = datetime.strptime(Date, "%m/%d/%Y")
        return date1.strftime("%Y-%m-%d")
    df.insert(11, "Date_new", df.apply(lambda row : Date_new(row["Date"]), axis=1))
    # Drop column: 'Date'
    df = df.drop(columns=['Date'])
    # Rename column 'Date_new' to 'Date'
    df = df.rename(columns={'Date_new': 'Date'})
    return df

# Loaded variable 'df' from URI: d:\GitHub_Desktop\Supermarket-Sales-SQL-Project\supermarket_sales_raw_data.csv
df = pd.read_csv(r'd:\GitHub_Desktop\Supermarket-Sales-SQL-Project\supermarket_sales_raw_data.csv')

df_clean = clean_data(df.copy())
df_clean.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.14,548.97,2019-01-05,13:08,Ewallet,522.83,4.76,26.14,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.82,80.22,2019-03-08,10:29,Cash,76.40,4.76,3.82,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.22,340.53,2019-03-03,13:23,Credit card,324.31,4.76,16.22,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.29,489.05,2019-01-27,20:33,Ewallet,465.76,4.76,23.29,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.21,634.38,2019-02-08,10:37,Ewallet,604.17,4.76,30.21,5.3


Create a database